In [ ]:
import numpy as np
import pandas as pd
from numpy import log2 as log
eps = np.finfo(float).eps
t_data = pd.read_csv("./dataset/train.csv")

In [ ]:
t_data = t_data.drop(['Name', 'Ticket', 'Cabin', 'PassengerId', 'Fare'], axis = 1)
t_data = t_data.dropna()

In [ ]:
t_data['Age'] = t_data['Age'].interpolate() #fixing missing values of age

In [ ]:
#Entropy of the entire dataset
entropy = 0
values = t_data['Survived'].unique()
for value in values:
    p = t_data['Survived'].value_counts()[value]/len(t_data.Survived)
    entropy = entropy + (-(p*log(p)))
print(entropy)

In [ ]:
def attribute_entropy(data, col):
    #print("Calculating att-ent")
    y = data['Survived'].unique()
    x = data[col].unique()
    entropy = 0
    for i in x:
        e = 0
        for j in y:
            num = len(data[col][data[col] == i][data['Survived'] == j])
            den = len(data[col][data[col] == i])
            p = num/(den+eps)
            e = e + (-(p*log(p+eps)))
        p2 = den/len(data)
        entropy = entropy + (-(p2*e))
    return abs(entropy)

In [ ]:
print(attribute_entropy(t_data, 'Age'))

In [ ]:
def find_max(d):
    #print("finding max")
    data = d.drop(['Survived'], axis = 1)
    Entropy_att = []
    IG = []
    #print(data.keys())
    for key in data.keys()[:-1]:
        IG.append(entropy-attribute_entropy(d, key))
    print(IG)
    return data.keys()[np.argmax(IG)]
find_max(t_data)

In [74]:
def get_subtable(df, node,value):
    print('making a subtable')
    return df[df[node] == value].reset_index(drop=True)


def buildTree(data): 
    
    node = find_max(data)
    
    vals = np.unique(data[node])
    #print(vals)
    tree = {}
    tree[node] = {}
    for val in vals:
        subtable = get_subtable(data, node, val)
        #print((subtable.head()))
        
        if(len(set(subtable['Survived'])) == 1):
            tree[node][val] = subtable['Survived'].iloc[0]
        else:
            tree[node][val] = buildTree(subtable)
    return tree

tree = buildTree(t_data)
print(tree)
  

[0.09400998456880671, 0.2141083128357235, 0.1272473570441669, 0.02486899311401769, 0.03068201178809904]
making a subtable
[0.3935639322525507, 0.16686288142080752, 0.431689213905875, 0.19844406116624813, 0.20877511570046214]
making a subtable
making a subtable
making a subtable
[0.43266691941192614, 0.05522316833019203, 0.05522316833019203, 0.30685233571801473, 0.5143710853574366]
making a subtable
[0.30685233571801473, 0.05522316833019203, 0.05522316833019203, 0.30685233571801473, 0.05522316833019203]
making a subtable
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.0264

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615

making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.02648099761531841

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615

...

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
making a subtable
[-0.026480997615318413, -0.026480997615318413, -0.026480997615

KeyboardInterrupt: 